# Process optimization by genetic algorithm

10 same items scheduling

In [2]:
include("../../src/ProcGA.jl")

Main.ProcGA

In [3]:
using CSV
using DelimitedFiles
using DataFrames
using Missings

## reading data

In [4]:
ptbl = CSV.read("proctable.csv",header = 1);

In [5]:
# take smaller size 
ptbl = ptbl[1:10,:];

In [6]:
mtbl = convert(Array, ptbl[1:2])

10×2 Array{Any,2}:
 "BB105 間管"       50
 "BB201-3 3補助管2"  50
 "BH301M 間管"      50
 "BH301M 枝管1"     50
 "BH301M 枝管2"     50
 "EP201 間管"       50
 "EP201-2 2補助管"   50
 "EP201-2 枝管1"    50
 "HR567 枝管4"      50
 "HR567 枝管4C"     50

In [7]:
ptbl2 = [coalesce.(x,0) for x in convert(Array, ptbl[3:end])]

10×14 Array{Int64,2}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [8]:
s = string.(names(ptbl)[3:end])

14-element Array{String,1}:
 "pipecut"           
 "anneal_bs_prebend" 
 "anneal_ns_prebend" 
 "icing"             
 "rotbend"           
 "pressbend"         
 "anneal_bs_precalib"
 "anneal_ns_precalib"
 "vcalib"            
 "hcalib"            
 "buff_precut"       
 "mc_cut"            
 "buff_postcut"      
 "qc_check"          

In [9]:
nametable = DataFrame(id = 1:length(s), nm = s)

,id,nm
,Int64,String
1,1,pipecut
2,2,anneal_bs_prebend
3,3,anneal_ns_prebend
4,4,icing
5,5,rotbend
6,6,pressbend
7,7,anneal_bs_precalib
8,8,anneal_ns_precalib
9,9,vcalib


Read grouping info.

In [10]:
gtbl = readdlm("grouping.csv",',', skipstart = 1)

2×4 Array{Any,2}:
 "anneal_bs"  "[2;7]"  300  3
 "anneal_ns"  "[3;8]"  300  3

In [11]:
gtbl2=[]
for k in 1:size(gtbl)[1]
    x = gtbl[k, 2:end]
    push!(gtbl2,Dict(:id=>eval(Meta.parse(x[1])),:cnt=>x[2],:timespan=>x[3]))
end

In [12]:
gtbl2

2-element Array{Any,1}:
 Dict{Symbol,Any}(:timespan=>3,:id=>[2, 7],:cnt=>300)
 Dict{Symbol,Any}(:timespan=>3,:id=>[3, 8],:cnt=>300)

## Initialize ProcGA tables.

In [13]:
ProcGA.settable(ptbl2, gtbl2, mtbl)

true

## Initialize population

In [14]:
g1 = ProcGA.jobtablebase()

10×120 Array{Int64,2}:
 1  7  7  7  9  9  12  12  13  13  14  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0

Read base data manually created from 6-same best practice.

In [17]:
bs = readdlm("10same.csv",',',Int)

10×35 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …   0   0   0   0   0  0  0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0      0  14  14   0   0  0  0   0   0
 0  0  1  0  7  7  7  9  9  12  12  13      0   0   0   0   0  0  0   0   0
 0  0  0  1  0  0  0  7  7   7   0   9      0   0   0   0   0  0  0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7     13   0   0  14  14  0  0   0   0
 0  0  0  0  0  1  0  7  7   7   0   0  …  14   0   0   0   0  0  0   0   0
 0  0  0  0  0  0  1  7  7   7   0   0      0   0   0   0   0  0  0   0   0
 0  0  0  0  0  0  0  1  0   0   7   7      0  13  13   0   0  0  0  14  14
 0  0  0  0  0  0  0  0  1   0   7   7      0   0   0   0   0  0  0   0   0
 0  0  0  0  0  0  0  0  0   1   0   0     13   0   0  14  14  0  0   0   0

In [18]:
ca = size(g1)[2]-size(bs)[2]

85

In [21]:
gbs = hcat(bs,zeros(Int,(10,ca)))

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [22]:
ProcGA.penalty(gbs)

35

In [23]:
ProcGA.checkvalidity(gbs)

false

In [24]:
ProcGA.validatejob!(gbs,10000)

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0  9  9  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [26]:
ProcGA.checkvalidity(gbs)

true

In [27]:
ProcGA.penalty(gbs)

102

Make population

In [28]:
ppl = ProcGA.initpopulationfrom(gbs,100);

In [29]:
vlst = []

0-element Array{Any,1}

## Evolution!

In [30]:
# Proceed generation
vl = ProcGA.evolution!(ppl, 500);

i:10 => (76, 77.0, 717)
i:20 => (76, 76.0, 83)
i:30 => (76, 76.0, 771)
i:40 => (76, 76.0, 111)
i:50 => (70, 76.0, 676)
i:60 => (70, 76.0, 657)
i:70 => (70, 76.0, 676)
i:80 => (70, 76.0, 655)
i:90 => (70, 76.0, 686)
i:100 => (70, 76.0, 740)
i:110 => (70, 75.0, 762)
i:120 => (69, 75.0, 755)
i:130 => (69, 74.0, 755)
i:140 => (69, 69.0, 96)
i:150 => (69, 69.0, 722)
i:160 => (68, 69.0, 95)
i:170 => (68, 69.0, 527)
i:180 => (68, 69.0, 711)
i:190 => (68, 69.0, 688)
i:200 => (68, 69.0, 69)
i:210 => (68, 69.0, 705)
i:220 => (68, 69.0, 658)
i:230 => (68, 69.0, 711)
i:240 => (68, 69.0, 724)
i:250 => (68, 69.0, 672)
i:260 => (68, 69.0, 711)
i:270 => (68, 68.0, 711)
i:280 => (68, 68.0, 663)
i:290 => (68, 68.0, 627)
i:300 => (68, 68.0, 629)
i:310 => (68, 68.0, 579)
i:320 => (68, 68.0, 694)
i:330 => (68, 68.0, 714)
i:340 => (68, 68.0, 118)
i:350 => (68, 68.0, 702)
i:360 => (68, 68.0, 668)
i:370 => (68, 68.0, 663)
i:380 => (68, 68.0, 88)
i:390 => (68, 68.0, 128)
i:400 => (68, 68.0, 663)
i:410 => (68, 

In [31]:
vlst = vcat(vlst,vl);

In [32]:
using Plots

┌ Info: Recompiling stale cache file C:\Users\hp\.julia\compiled\v1.0\Plots\ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1190


Pick up the best data.

In [49]:
v = ProcGA.penalty.(ppl)
idm = findmin(v)[2]
bestgen = ppl[idm]

6×72 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  0  0  0  0  0   9   9   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7  …  0  0  0  0  0  0  0  0  0  0  0  0

In [50]:
ProcGA.checkvalidity(bestgen)

true

In [51]:
ProcGA.penalty(bestgen)

31

In [52]:
ProcGA.clipjob(bestgen)

6×31 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …   0   0   0   0   0   0   0   0   0
 1  7  7  7  0  0  0  0  0   9   9   0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  1  0  7  7   7   0   0      0  13  13  14  14   0   0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0     13   0   0   0   0  14  14   0   0
 0  0  0  1  0  0  0  7  7   7   0   9      0  14  14   0   0   0   0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7  …  12   0   0  13  13   0   0  14  14

In [54]:
ProcGA.validlength(bestgen)

31

Sort it by first appearance of process.

In [69]:
bestgensort = similar(bestgen)
for i in 1:size(bestgen)[1]
    idx = findfirst(x->(x>0),bestgen[i,:])
    bestgensort[idx,:] = bestgen[i,:]
end
bestgensort

6×72 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0

In [70]:
# save best process table to file.
# do not clip it so that it can be used again
writedlm("bestprc_20181112.csv",bestgensort, ',')

## read from saved data

In [40]:
rdlst = readdlm("bestprc_20181112.csv", ',', Int)

6×72 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  0  0  0  0  0   9   9   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7  …  0  0  0  0  0  0  0  0  0  0  0  0

In [41]:
ppl = ProcGA.initpopulationfrom(rdlst,100);

## Conclusion

For given 6 same data with proctalbe below, the best combination is "bestgen" here.

Its length is 31.
Taking time unit as 1 hour and assuming job time as 7 hours/day, it needs almost 4 days to finish these materials.

In [58]:
transpose(ProcGA.proctable[1,:])

1×14 LinearAlgebra.Transpose{Int64,Array{Int64,1}}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [59]:
ProcGA.clipjob(bestgen)

6×31 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …   0   0   0   0   0   0   0   0   0
 1  7  7  7  0  0  0  0  0   9   9   0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  1  0  7  7   7   0   0      0  13  13  14  14   0   0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0     13   0   0   0   0  14  14   0   0
 0  0  0  1  0  0  0  7  7   7   0   9      0  14  14   0   0   0   0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7  …  12   0   0  13  13   0   0  14  14